# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Overview of the dataset
print(data.head())

# Count the number of viewers and clickers
viewers = data[data['action'] == 'view']['id'].nunique()
clickers = data[data['action'] == 'click']['id'].nunique()

print(f"Number of viewers: {viewers}")
print(f"Number of clickers: {clickers}")

# Check for anomalies: Anyone clicked without viewing?
clicked_without_viewing = data[(data['action'] == 'click') & ~(data['id'].isin(data[data['action'] == 'view']['id']))]
if not clicked_without_viewing.empty:
    print("Anomalies: Some users clicked without viewing.")
    print(clicked_without_viewing)

# Check for overlap between control and experiment groups
control_users = data[data['group'] == 'control']['id'].unique()
experiment_users = data[data['group'] == 'experiment']['id'].unique()
overlap = set(control_users).intersection(experiment_users)

print(f"Number of users in control group: {len(control_users)}")
print(f"Number of users in experiment group: {len(experiment_users)}")
print(f"Number of overlapping users: {len(overlap)}")

                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Number of viewers: 6328
Number of clickers: 1860
Number of users in control group: 3332
Number of users in experiment group: 2996
Number of overlapping users: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [4]:
#Your code here
import statsmodels.api as sm
import statsmodels.stats.proportion as proportion

# Calculate the number of clicks and total users in the control and experiment groups
control_clicks = data[(data['group'] == 'control') & (data['action'] == 'click')]['id'].nunique()
experiment_clicks = data[(data['group'] == 'experiment') & (data['action'] == 'click')]['id'].nunique()

control_users = data[data['group'] == 'control']['id'].nunique()
experiment_users = data[data['group'] == 'experiment']['id'].nunique()

# Calculate click-through rates (CTR) for both groups
control_ctr = control_clicks / control_users
experiment_ctr = experiment_clicks / experiment_users

# Perform a two-sample proportion z-test
z_score, p_value = proportion.proportions_ztest([control_clicks, experiment_clicks], [control_users, experiment_users])

# Set the significance level (alpha)
alpha = 0.05

# Check if the p-value is less than alpha to determine statistical significance
if p_value < alpha:
    print("Reject the null hypothesis: The experimental homepage is more effective.")
else:
    print("Fail to reject the null hypothesis: No significant difference.")

Reject the null hypothesis: The experimental homepage is more effective.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [6]:
#Your code here
expected_clicks_experiment = control_ctr * experiment_users
expected_clicks_experiment

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [7]:
#Your code here
# Calculate the standard error for the binomial distribution
standard_error = (control_ctr * (1 - control_ctr) / control_users) ** 0.5

# Calculate the z-score
z_score = (experiment_clicks - expected_clicks_experiment) / standard_error

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [8]:
#Your code here
p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.